In [1]:
from datetime import datetime
from docx import Document
import pandas as pd
import numpy as np
import pyodbc

In [2]:
#Define the connection Parameters
# con= pyodbc.connect("DRIVER={ODBC DRIVER 17 for SQL Server};Server= server;DATABASE=dbName;UI=userName;PWD=pwd;")
server = 'uzima-dmac.database.windows.net'
dbName ='Uzima_db'
user='dorcas.mwigereri@ea.aku.edu'
Authentication='ActiveDirectoryInteractive'
driver= '{ODBC Driver 17 for SQL Server}'
conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+dbName+';UID='+user+';AUTHENTICATION='+Authentication)

In [3]:
#SQL Queries to load the views
sql_data_query="SELECT * from [dbo].[DepressionAnalysis]"
enrollment_data = "SELECT * from [DW].[DimEnrolledParticipants]"
#Read Data
data = pd.read_sql(sql_data_query,conn)
enrollment_data = pd.read_sql(enrollment_data,conn)
pivoted_data =data.pivot(index=['participantidentifier'], columns='resultidentifier', values='answers').reset_index()
enrollment_df=enrollment_data[['ParticipantIdentifier','EnrollmentDate']]
#Calculate the Age for each Participant based on Participant's enrolled date and Participants DOB
#Merge pivotted data with the enrollment data
pivoted_data = pd.merge(pivoted_data,enrollment_df, left_on='participantidentifier', right_on='ParticipantIdentifier')
pivoted_data['DOB'] = pd.to_datetime(pivoted_data['DOB'])    
# Get today's date
pivoted_data['EnrollmentDate'] = pd.to_datetime(pivoted_data['EnrollmentDate'])     
# Calculate age 
pivoted_data['Age'] = pivoted_data.apply(lambda row: row['EnrollmentDate'].year - row['DOB'].year - 
                     ((row['EnrollmentDate'].month, row['EnrollmentDate'].day) < (row['DOB'].month, row['DOB'].day)), axis=1)
    

# #Calculate the PHQ9 Value and add it as a column to the dataframe
phq9_columns = ['down0', 'interest0', 'asleep0', 'tired0', 'appetite0', 'failure0', 'concentr0', 'activity0', 'suic0']
pivoted_data[phq9_columns] = pivoted_data[phq9_columns].apply(pd.to_numeric, errors='coerce')
pivoted_data['PHQ9'] = pivoted_data[phq9_columns].sum(axis=1)
pivoted_data.to_excel("C:\\Users\\dorcas.mwigereri\\.conda\\envs\\HCW_Study\\Depression Paper - Andrew\Depression paper\\Data\\SurveyResultsPivoted.xlsx", sheet_name='Sheet1', index=False)
# pivoted_data.head()


C:\Users\dorcas.mwigereri\AppData\Local\Temp\6\ipykernel_14112\2180066906.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql_data_query,conn)
C:\Users\dorcas.mwigereri\AppData\Local\Temp\6\ipykernel_14112\2180066906.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  enrollment_data = pd.read_sql(enrollment_data,conn)


PHQ9 Analysis


In [4]:
#Proportion of PhQ9 for the total Population

count_below_10 = (pivoted_data['PHQ9'] >= 0) & (pivoted_data['PHQ9'] < 10)
count_above_10 = (pivoted_data['PHQ9'] >= 10)

# Create a new Document
phq9_proportion_doc = Document()

# Add headings
phq9_proportion_doc.add_heading('PHq9 Counts', level=1)
phq9_proportion_doc.add_heading('Counts between 0-9', level=2)
phq9_proportion_doc.add_paragraph(str(count_below_10.sum()))

phq9_proportion_doc.add_heading('Counts 10 and above', level=2)
phq9_proportion_doc.add_paragraph(str(count_above_10.sum()))

# Save the document
phq9_proportion_doc.save("C:\\Users\\dorcas.mwigereri\\.conda\\envs\\HCW_Study\\Depression Paper - Andrew\Depression paper\\PHQ9_Analysis\\PHQ9_proportions.docx")

In [5]:
def categorize_phq9(score):
    if 0 <= score <= 9:
        return '0-9'
    elif score >= 10:
        return '10 and above'
    else:
        return 'Invalid'  # in case of any invalid scores




PHQ9 PROPORTIONS 

In [6]:
#Apply the function to create a new column 'PHQ9_Category'
pivoted_data['PHQ9_Category'] = pivoted_data['PHQ9'].apply(categorize_phq9)

#Create Function to generate a dicument with the Counts and Percentage summaries based on the PHQ9 categories
def calculate_phq9_proportion(data,dfName):
       
     #Create a new Document
    doc = Document()

    # # Add title
    doc.add_heading('PHQ9 Analysis Summary', level=1)
    for category_name in data:
        summary_value =pivoted_data.groupby([category_name,'PHQ9_Category']).size().unstack(fill_value=0)
        
        # summary_percentage =summary_value   
        total_counts = summary_value.values.sum()
        percentage_summary =(summary_value/total_counts)*100     
       
       # Add table to document
        def add_table(doc, title, df, df_percentage):
            doc.add_heading(title, level=2)
            table = doc.add_table(rows=1, cols=(len(df.columns) * 2) + 1)
            hdr_cells = table.rows[0].cells
            hdr_cells[0].text = category_name
            
            # Set headers for counts and percentages
            for i, col in enumerate(df.columns):
                hdr_cells[(i * 2) + 1].text = f'{col} Count'
                hdr_cells[(i * 2) + 2].text = f'{col} %'
            
            # Fill rows with data
            for idx in df.index:
                row_cells = table.add_row().cells
                row_cells[0].text = str(idx)
                for i, col in enumerate(df.columns):
                    row_cells[(i * 2) + 1].text = str(df.loc[idx, col])
                    row_cells[(i * 2) + 2].text = f"{df_percentage.loc[idx, col]:.2f}%"

        add_table(doc, f'Summary by {category_name}', summary_value, percentage_summary)
    
    #Save the document    
    doc.save(rf'C:\Users\dorcas.mwigereri\.conda\envs\HCW_Study\Depression Paper - Andrew\Depression paper\PHQ9_Analysis\PHQ9_{dfName}_Analysis_Summary.docx')



In [7]:
#create a lis of each section requiring the counts and percentage summaries

demographics_df =['Gender','Age','Marital','Children','Cadre','Othercadre','Experience','Education']
discrimination_df =['Discrimination']
SLE_df=['Events','Pressuremarry','Events1','Unwantedpregnancy','Event3','Unwantedchild','Event4']
suicidal_df=['Dead']
medical_error_df =['Majorerror','Errortype','Othererror']
scary_events_df=['Disaster','Accident','Robbed','Beatfamily','Beatnotfamily','Familybeat','Commbeat','Touch','Pressure','Dying','Attacked','Someoneattack','Stressfulmed','War','Otherstressful','Otherevent']
workplace_df =['Hoursduty','Hoursworked']

#Call function to create PHQ9 summaries for the counts and percentages
calculate_phq9_proportion(demographics_df,"Demography_Data")
calculate_phq9_proportion(discrimination_df,"Discimination_Data")
calculate_phq9_proportion(SLE_df,"SLE_Data")
calculate_phq9_proportion(suicidal_df,"Suicidal_Data")
calculate_phq9_proportion(medical_error_df,"Medical_Error_Data")
calculate_phq9_proportion(scary_events_df,"Scary_Events_Data")
calculate_phq9_proportion(workplace_df,"Workplace_Data")